## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import re

from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True )


from data_preprocess import DataTransformer

random_state = 12345
NUM_ITERATIONS = 5000

FEATURES = 'base feature'

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [3]:
# читаем исходные датасеты и складываем в один
train_ds = pd.read_csv('data/train_dataset.csv')
test_ds = pd.read_csv('data/test_dataset.csv')
train_ds = pd.concat([train_ds, test_ds])

# запоминаем дату начала тестовых данных, потом также поступим и с закрытым датасетом
open_test_begin = pd.to_datetime(test_ds['date']).min()
open_test_end = pd.to_datetime(test_ds['date']).max() + pd.to_timedelta(1,'d')
print('начало открытого теста:', open_test_begin, '    конец открытого теста:', open_test_end)

начало открытого теста: 2023-04-01 00:00:00     конец открытого теста: 2023-08-01 00:00:00


In [4]:
train_ds.columns

Index(['date', 'time', 'target', 'temp', 'temp_pred', 'weather_pred',
       'weather_fact'],
      dtype='object')

In [5]:
transformer = DataTransformer()

In [6]:
train_ds = transformer.transform(train_ds)

FileNotFoundError: [Errno 2] No such file or directory: 'data/holidays_true.csv'

#### 1.11 Исключение лишних колонок

In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] #, 'temp_pred'] #, 'target_lag_336'] 

for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

In [ ]:
# Формируем набор датасетов для обучения и проверки
features = train_ds[feature_cols]
target = train_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    return features_interval, target_interval


# для проверки на тестовой выборке будем учиться на всем тренировочном датасете

features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)


features_open_test_sum, target_open_test_sum = features_interval(features, target, '2023-06-01', open_test_end)



FEATURES = 'test_summer_fulltest'



### 2. Обучение моделей

В настоящей работе обучается модель LightGBM

#### 2.1 Гиперпараметры
Были подобраны следующие значения гиперпараметров:

In [ ]:
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':1, 'num_iterations':NUM_ITERATIONS, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

## XGBoost build

In [ ]:
#features_train, target_train
#features_valid, target_valid

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

### 4. Проверка метрик на тестовом датасете

#### 4.1 LGBM

In [ ]:
features_all_train.columns

In [ ]:
drop_list = [#'preholidays',
            #'has_rain_probability', 'W', 'E',
            #'holidays' 
            ]
feat_lgbm_train = features_all_train.drop(columns=drop_list)
feat_lgbm_test = features_open_test.drop(columns=drop_list)
feat_lgbm_train.columns, feat_lgbm_test.columns

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете

lgbm_model_all_train = lgb.Booster(model_file='models/lgb_model.txt')

l_predict_train = lgbm_model_all_train.predict(feat_lgbm_train)
l_predict_test = lgbm_model_all_train.predict(feat_lgbm_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, l_predict_test)

results = pd.DataFrame([[f'тренировочная LGBM {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете




l_predict_test = lgbm_model_all_train.predict(features_open_test_sum)


mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test_sum, l_predict_test)

results = pd.concat([results,
pd.DataFrame([ [f'тестовая LGBM_only summer {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
results

#### 4.2 XGBoost

In [ ]:
features_all_train.columns

In [ ]:
drop_list = ['preholidays_true',
            'has_rain_probability', 
            #'holidays', 'preholidays',
             'W', 'E'
            ]
feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
feat_xgb_train.columns, feat_xgb_test.columns

In [ ]:
drop_list = ['preholidays_true',
            #'has_rain_probability', 
            # 'W', 'E'
            ]
n_values = range(1, 24)
preholidays = ['preholidays_true_{}'.format(n) for n in n_values]
#has_rain = ['has_rain_probability_{}'.format(n) for n in n_values]
#W_wind = ['W_{}'.format(n) for n in n_values]
#E_wind = ['E_{}'.format(n) for n in n_values]

drop_list = drop_list + preholidays #+ has_rain + W_wind + E_wind

feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
feat_xgb_train.columns, feat_xgb_test.columns

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете


xgb_model_all_train = XGBRegressor()

# загружаем модель из файла
xgb_model_all_train.load_model('models/xgb_modelbase feature_3.json')

xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)
xgb_predict_train = xgb_model_all_train.predict(feat_xgb_train)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])


In [ ]:
xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)


mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])


In [ ]:
results

### 5. Ensemble

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor

import optuna
from optuna.samplers import TPESampler

In [ ]:
results_ensemble = pd.DataFrame(columns=('Выборка', 'MAE', 'MAPE', 'R2'))

### Simple Ensemble

In [ ]:
predict_simple_ensemble_train = (xgb_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (xgb_predict_test + l_predict_test)/2



In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)

results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная simple_ensemble  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая simple_ensemble {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
display(results_ensemble)
results

In [ ]:
mae_day(target_open_test, predict_simple_ensemble_test)

import os

# определите путь к папке, которую вы хотите создать
folder_path = "models"

# проверьте, существует ли уже папка
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

lgbm_model_all_train.booster_.save_model('models/lgb_model.txt')
xgb_model.save_model('models/xgb_model.json')

In [ ]:
results.to_csv(f'models/results_LGBM_XGBoost_simple_ensemble_{FEATURES}')